In [1]:
# SELF REMINDER. Copy the 'ocpa' directory to the forked one from github, so that I can push updates to github.
# Python native
import pickle
from statistics import median, mean
import pandas as pd
# Data handling
# Object centric process mining
# import ocpa.algo.evaluation.precision_and_fitness.utils as evaluation_utils # COMMENTED OUT BY TIM
# import ocpa.algo.evaluation.precision_and_fitness.evaluator as precision_fitness_evaluator # COMMENTED OUT BY TIM
import ocpa.objects.log.importer.csv.factory as csv_import_factory
import ocpa.algo.predictive_monitoring.factory as feature_factory
from ocpa.objects.log.ocel import OCEL

# # Simple machine learning models, procedure tools, and evaluation metrics
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer

# # Custom GNN tools
# from gnn_utils import (
#     generate_graph_dataset,
#     # get_ordered_event_list,
#     # visualize_graph,
#     # show_remaining_times,
#     # visualize_instance,
#     # GraphDataLoader,
#     # GCN,
#     # evaluate_gnn,
# )

# PyG
import torch
from replicating.ocpa_PyG_integration.EventGraphDataset import EventGraphDataset
from replicating.ocpa_PyG_integration.EventSubGraphDataset import EventSubGraphDataset

# Global variables
from replicating.experiment_config import STORAGE_PATH, RANDOM_SEED, TARGET_LABEL

filename = "data/adams/example_logs/mdl/BPI2017-Final.csv"
object_types = ["application", "offer"]
parameters = {
    "obj_names": object_types,
    "val_names": [],
    "act_name": "event_activity",
    "time_name": "event_timestamp",
    "sep": ",",
}
file_path_object_attribute_table = None

c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.13.1+cpu
Cuda available: False
Torch geometric version: 2.2.0
Torch version: 1.13.1+cpu
Cuda available: False
Torch geometric version: 2.2.0


In [2]:
ds_train = EventGraphDataset(
    root=STORAGE_PATH,
    filename="BPI2017-adams.fs",
    label_key=TARGET_LABEL,
    size_subgraph_samples=4,
    train=True,
    verbosity=51
)
ds_val = EventGraphDataset(
    root=STORAGE_PATH,
    filename="BPI2017-adams.fs",
    label_key=TARGET_LABEL,
    size_subgraph_samples=4,
    validation=True,
    verbosity=51
)
ds_test = EventGraphDataset(
    root=STORAGE_PATH,
    filename="BPI2017-adams.fs",
    label_key=TARGET_LABEL,
    size_subgraph_samples=4,
    test=True,
    verbosity=51
)

In [7]:
ds_val.get_summary()

EventSubGraphDataset (#graphs=4):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |       40 |     47.5 |
| std        |        0 |      0.6 |
| min        |       40 |     47   |
| quantile25 |       40 |     47   |
| median     |       40 |     47   |
| quantile75 |       40 |     48   |
| max        |       40 |     48   |
+------------+----------+----------+

In [ ]:
# Importing OCEL
ocel = csv_import_factory.apply(
    filename, csv_import_factory.TO_OCEL, parameters, file_path_object_attribute_table
)

In [12]:
with open(f"debug/self._mapping-pass.pkl", "rb") as file:
    mapping_pass = pickle.load(file)
with open("debug/attribute-pass.pkl", "rb") as file:
    attribute_pass = pickle.load(file)
with open("debug/e_id-pass.pkl", "rb") as file:
    e_id_pass = pickle.load(file)
with open("debug/self._mapping[attribute][e_id]-pass.pkl", "rb") as file:
    value_pass = pickle.load(file)

with open(f"debug/self._mapping-fail.pkl", "rb") as file:
    mapping_fail = pickle.load(file)
with open("debug/attribute-fail.pkl", "rb") as file:
    attribute_fail = pickle.load(file)
with open("debug/e_id-fail.pkl", "rb") as file:
    e_id_fail = pickle.load(file)

with open("debug/ocel.log", "rb") as file:
    ocel_log = pickle.load(file)

In [13]:
map = pd.DataFrame(ocel.log._mapping)
map.head()

,application,offer,event_id,event_activity,event_timestamp,event_start_timestamp
0,[Application_652823628],[],0,Create application,2016-01-01 10:51:15.304,2016-01-01 10:51:15.304
1,[Application_652823628],[],1,Submit,2016-01-01 10:51:15.352,2016-01-01 10:51:15.352
2,[Application_652823628],[],2,Complete,2016-01-02 11:45:22.429,2016-01-02 11:45:22.429
3,[Application_652823628],[],3,Accept,2016-01-02 12:23:04.299,2016-01-02 12:23:04.299
4,[Application_652823628],[Offer_148581083],4,Create offer,2016-01-02 12:29:05.354,2016-01-02 12:29:05.354


In [20]:
ocel.parameters

{'obj_names': ['application', 'offer'],
 'val_names': [],
 'act_name': 'event_activity',
 'time_name': 'event_timestamp',
 'sep': ','}

In [28]:
mapping_pass = pd.DataFrame(mapping_pass)
print(attribute_pass)
print(e_id_pass)
print(value_pass)
mapping_pass.head()

event_timestamp
116864
2016-05-03 15:02:10.255000


,application,offer,event_id,event_activity,event_timestamp,event_start_timestamp,event_objects
0,[Application_652823628],[],0,Create application,2016-01-01 10:51:15.304,2016-01-01 10:51:15.304,"[(application, Application_652823628)]"
1,[Application_652823628],[],1,Submit,2016-01-01 10:51:15.352,2016-01-01 10:51:15.352,"[(application, Application_652823628)]"
2,[Application_652823628],[],2,Complete,2016-01-02 11:45:22.429,2016-01-02 11:45:22.429,"[(application, Application_652823628)]"
3,[Application_652823628],[],3,Accept,2016-01-02 12:23:04.299,2016-01-02 12:23:04.299,"[(application, Application_652823628)]"
4,[Application_652823628],[Offer_148581083],4,Create offer,2016-01-02 12:29:05.354,2016-01-02 12:29:05.354,"[(application, Application_652823628), (offer,..."


In [30]:
mapping_fail = pd.DataFrame(mapping_fail)
print(attribute_fail)
print(e_id_fail)
mapping_fail.head()

event_RequestedAmount
116864


,application,offer,event_id,event_activity,event_timestamp,event_start_timestamp,event_objects
0,[Application_652823628],[],0,Create application,2016-01-01 10:51:15.304,2016-01-01 10:51:15.304,"[(application, Application_652823628)]"
1,[Application_652823628],[],1,Submit,2016-01-01 10:51:15.352,2016-01-01 10:51:15.352,"[(application, Application_652823628)]"
2,[Application_652823628],[],2,Complete,2016-01-02 11:45:22.429,2016-01-02 11:45:22.429,"[(application, Application_652823628)]"
3,[Application_652823628],[],3,Accept,2016-01-02 12:23:04.299,2016-01-02 12:23:04.299,"[(application, Application_652823628)]"
4,[Application_652823628],[Offer_148581083],4,Create offer,2016-01-02 12:29:05.354,2016-01-02 12:29:05.354,"[(application, Application_652823628), (offer,..."


In [ ]:
try:
            self._mapping[attribute][e_id]
            with open("self._mapping-pass.pkl", "wb") as file:
                pickle.dump(self._mapping, file)
            with open("attribute-pass.pkl", "wb") as file:
                pickle.dump(attribute, file)
            with open("e_id-pass.pkl", "wb") as file:
                pickle.dump(e_id, file)
            with open("self._mapping[attribute][e_id]-pass.pkl", "wb") as file:
                pickle.dump(self._mapping[attribute][e_id], file)
        except:
            with open("self._mapping-fail.pkl", "wb") as file:
                pickle.dump(self._mapping, file)
            with open("attribute-fail.pkl", "wb") as file:
                pickle.dump(attribute, file)
            with open("e_id-fail.pkl", "wb") as file:
                pickle.dump(e_id, file)